<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [128]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [129]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [130]:
apps = apps.drop(columns=['Type', 'Price', 'Genres', 'Last Updated', 'Current Ver'])

In [131]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})
apps = apps.drop_duplicates(subset=['App'], keep='first')

C:\Users\elere\AppData\Local\Temp\ipykernel_25108\2542957938.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_25108\2542957938.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_25108\2542957938.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [132]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [133]:
DataProcessing.shuffle(apps)

In [134]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.95)
validatingSet = validatingSet.reset_index(drop=True)
DataProcessing.normalization(validatingSet)
trainingSet = trainingSet.reset_index(drop=True)
DataProcessing.normalization(trainingSet)
validatingSet

,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver
0,BR Video Player,VIDEO_PLAYERS,0.750,0.000007,0.002378,0.000050,Everyone,30.0
1,CM Security Lite - Antivirus,TOOLS,0.900,0.042280,0.000119,0.100000,Everyone,23.0
2,Neuvaine au Saint-Esprit,LIFESTYLE,0.925,0.000009,0.005589,0.000100,Everyone,40.0
3,Casa CF,HEALTH_AND_FITNESS,0.925,0.000002,0.006184,0.000005,Everyone,41.0
4,Ghost In Photo,PHOTOGRAPHY,0.725,0.004586,0.015341,0.050000,Teen,23.0
...,...,...,...,...,...,...,...,...
357,RAM Cleanup Ad-Free Option,TOOLS,0.850,0.000074,0.000000,0.000100,Everyone,41.0
358,Multi Surgery ER Emergency Hospital : Doctor Game,FAMILY,0.775,0.000036,0.080747,0.000100,Teen,40.0
359,Multiple Videos at Same Time,VIDEO_PLAYERS,0.850,0.000759,0.000476,0.010000,Everyone,44.0
360,MakeMyTrip-Flight Hotel Bus Cab IRCTC Rail Boo...,TRAVEL_AND_LOCAL,0.825,0.096776,0.042860,0.100000,Everyone,44.0


<h2>k-nearest neighbors algorithm</h2>

In [135]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        return math.sqrt(math.pow(v1[2] - v2[2], 2) + math.pow(v1[3] - v2[3], 2) + math.pow(v1[4] - v2[4], 2) + math.pow(v1[5] - v2[5], 2))
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        
            
        return dataset.head(10)

<h2>Apps Recommendation</h2>

In [136]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
avgApp = pd.DataFrame(columns=['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Content Rating', 'Android Ver'])
avgApp.loc[0] = ['a', 'b', avgRating, avgReviews, avgSize, avgInstalls, 'c', 'd']
recommendedApps = KNN.clustering(trainingSet, avgApp.loc[0])
recommendedApps

,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver,distance
891,Tom's Love Letters,FAMILY,0.775,0.015722,0.034240,0.05,Everyone,41.0,0.021378
987,Angry Birds Friends,GAME,0.800,0.018483,0.047331,0.05,Everyone,41.0,0.024389
1079,Talking Ginger 2,ENTERTAINMENT,0.800,0.015659,0.048338,0.05,Everyone,41.0,0.024518
3422,Draw Something Classic,GAME,0.800,0.024328,0.042296,0.05,Everyone,44.0,0.025319
1839,Adobe Photoshop Express:Photo Editor Collage M...,PHOTOGRAPHY,0.800,0.020378,0.036395,0.05,Everyone,44.0,0.025581
5127,free video calls and chat,COMMUNICATION,0.800,0.013248,0.036395,0.05,Everyone,44.0,0.025597
5701,Love Balls,GAME,0.800,0.008033,0.039275,0.05,Everyone,41.0,0.026152
2859,"The Voice, sing and connect",FAMILY,0.775,0.009040,0.036395,0.01,Everyone,42.0,0.026620
1180,Truck Driver Cargo,GAME,0.775,0.005737,0.043303,0.01,Everyone 10+,41.0,0.026682
6247,Real Football,SPORTS,0.775,0.013044,0.032226,0.01,Everyone,403.0,0.027275


<h2>KD Tree Algorithm</h2>

In [137]:
class Node:
    def __init__(self, point, left=None, right=None, name=None):
        self.point = point
        self.left = left
        self.right = right
        self.name = name


class KDTree:
    def __init__(self, points):
        self.k = 4  # Number of dimensions
        self.root = self.build_tree(points)

    def build_tree(self, points, depth=0):
        if len(points) == 0:
            return None

        # Select axis based on depth
        axis = depth % self.k

        # Sort points based on the axis
        sorted_points = points.sort_values(by=points.columns[axis+2])

        # Find the median
        median_index = len(sorted_points) // 2
        median = sorted_points.iloc[median_index]
        
        # Recursively build subtrees
        left_points = sorted_points[:median_index]
        right_points = sorted_points[median_index + 1:]

        left = self.build_tree(left_points, depth + 1)
        right = self.build_tree(right_points, depth + 1)
        return Node(median[2:6], left, right, median[0])

    def find_nearest_neighbors(self, query_point, k=1):
        nearest_points = []
        distances = []
        nearest_names = []

        def search(node, depth=0):
            if node is None:
                return

            # Select axis based on depth
            axis = depth % self.k

            # Traverse left or right subtree based on query point
            if query_point[axis] < node.point[axis]:
                search(node.left, depth + 1)
            else:
                search(node.right, depth + 1)

            # Calculate distance between query point and current point
            distance = np.linalg.norm(np.array(query_point) - np.array(node.point))
            # If the number of nearest points is less than k, add current point
            if len(nearest_points) == 0:
                distances.append(distance)
                nearest_points.append(node.point)
                nearest_names.append(node.name)
            
            elif len(nearest_points) < k:
                index = len(nearest_points)
                for i in range(0, len(nearest_points)):
                    if(distance < distances[i]):
                        index = i
                        break
                distances.insert(index, distance)
                nearest_points.insert(index, node.point)
                nearest_names.insert(index, node.name)
            else:
                # If the current point is closer than the farthest nearest point,
                # replace the farthest point with the current point
                if distance < distances[-1]:
                    for i in range(0, k):
                        if(distance < distances[i]):
                            distances.insert(i, distance)
                            nearest_points.insert(i, node.point)
                            nearest_names.insert(i, node.name)
                            break
                    distances.pop()
                    nearest_points.pop()
                    nearest_names.pop()
            # Check if there can be closer points on the other side of the splitting axis
            if abs(query_point[axis] - node.point[axis]) < distances[-1] or len(nearest_points) < k:
                if query_point[axis] < node.point[axis]:
                    search(node.right, depth + 1)
                else:
                    search(node.left, depth + 1)
        search(self.root)
        return nearest_names

<h2>Building the KD Tree</h2>

In [138]:
kdtree = KDTree(trainingSet)

In [139]:
#average app
query_point = avgApp[['Rating', 'Reviews', 'Size', 'Installs']].to_numpy()[0]

nearest_neighbors = kdtree.find_nearest_neighbors(query_point, k=5)

print("Nearest neighbors:")
for neighbor in nearest_neighbors:
    print(neighbor)

Nearest neighbors:
Tom's Love Letters
Angry Birds Friends
Talking Ginger 2
Draw Something Classic
Adobe Photoshop Express:Photo Editor Collage Maker
